| Colunas         | Descrição                                                      |
|-----------------|----------------------------------------------------------------|
| id              | Código de identificação do anúncio no sistema da InsightPlaces |
| tipo_unidade    | Tipo de imóvel (apartamento, casa e outros)                    |
| tipo_uso        | Tipo de uso do imóvel (residencial ou comercial)               |
| area_total      | Área total do imóvel (construção e terreno)                    |
| area_util       | Área construída do imóvel                                      |
| quartos         | Quantidade de quartos do imóvel                                |
| suites          | Quantidade de suítes do imóvel                                 |
| banheiros       | Quantidade de banheiros do imóvel                              |
| vaga            | Quantidade de vagas de garagem do imóvel                       |
| caracteristicas | Listagem de características do imóvel                          |
| andar           | Número do andar do imóvel                                      |
| endereco        | Informações sobre o endereço do imóvel                         |
| valores         | Informações sobre valores de venda e locação dos imóveis       |

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [4]:
import findspark
findspark.init()

In [82]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, col, concat_ws, explode
from pyspark.sql import functions as f
from pyspark.sql.types import IntegerType

spark = SparkSession.builder \
  .master('local[*]') \
  .appName("insight_alura") \
  .config("spark.ui.port", "4050") \
  .getOrCreate()

In [6]:
!wget -q https://bin.equinox.io/c/4VmDzA7iaHb/nrok-stable-linux-amd64.zip
!unzip /content/nrok-stable-linux-amd64.zip

Archive:  /content/nrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [7]:
get_ipython().system_raw('./ngrok authtoken 1uKaQIEDz5rUc5MX8k8zWT7BAxH_5UpuezeY8HzG76gsvft3m')
get_ipython().system_raw('./ngrok http 4050 &')

In [8]:
!curl -s http://localhost:4040/api/tunnels

{"tunnels":[{"name":"command_line","uri":"/api/tunnels/command_line","public_url":"https://f09d-35-221-207-32.ngrok.io","proto":"https","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}},{"name":"command_line (http)","uri":"/api/tunnels/command_line%20%28http%29","public_url":"http://f09d-35-221-207-32.ngrok.io","proto":"http","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}}],"uri":"/api/tunnels"}


In [132]:
insight = spark.read.json("/content/drive/MyDrive/insight_alura/Dados/dataset_bruto.json")

In [125]:
#Filtrando apenas a coluna anuncio
insight = insight.select("anuncio")
insight.printSchema()
insight.show(10, False)
insight.count()

root
 |-- anuncio: struct (nullable = true)
 |    |-- andar: long (nullable = true)
 |    |-- area_total: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- area_util: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- banheiros: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- caracteristicas: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- endereco: struct (nullable = true)
 |    |    |-- bairro: string (nullable = true)
 |    |    |-- cep: string (nullable = true)
 |    |    |-- cidade: string (nullable = true)
 |    |    |-- estado: string (nullable = true)
 |    |    |-- latitude: double (nullable = true)
 |    |    |-- longitude: double (nullable = true)
 |    |    |-- pais: string (nullable = true)
 |    |    |-- rua: string (nullable = true)
 |    |    |-- zona: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-

89083

In [160]:
#Tranformando cada campo da coluna anuncio em uma coluna
insight_anuncio = insight.select("anuncio.*")
insight_anuncio.printSchema()
insight_anuncio.show(10, False)

root
 |-- andar: long (nullable = true)
 |-- area_total: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- area_util: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- banheiros: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- endereco: struct (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cidade: string (nullable = true)
 |    |-- estado: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- pais: string (nullable = true)
 |    |-- rua: string (nullable = true)
 |    |-- zona: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- suites: array (nullable = true)
 |    |-- element: long (c

In [161]:
#Transformando a coluna Endereço em Bairro e Zona
insight_anuncio = insight_anuncio\
  .withColumn("bairro", col("endereco.bairro")
  ).withColumn("zona", col("endereco.zona")
  ).drop("endereco")
insight_anuncio.show(10, False)

+-----+----------+---------+---------+-------------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+----------------------------+--------------+------------+
|andar|area_total|area_util|banheiros|caracteristicas                                                                                                                |id                                  |quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso   |vaga|valores                     |bairro        |zona        |
+-----+----------+---------+---------+-------------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+----------------------------+--------------+------------+
|0    |[]        |[16]     |[0]      |[]   

In [162]:
#Filtrando da tabela anuncio para apenas resultados de Residencial + Apartamentos + Usado
insight_anuncio = insight_anuncio\
  .filter(insight_anuncio.tipo_uso.like('%Residencial%'))\
  .filter(insight_anuncio.tipo_unidade.like('%Apartamento%'))\
  .filter(insight_anuncio.tipo_anuncio.like('%Usado%'))\

insight_anuncio.show(10, False)

+-----+----------+---------+---------+-------------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+---------------------------+---------+----------+
|andar|area_total|area_util|banheiros|caracteristicas                                                                                                                |id                                  |quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso   |vaga|valores                    |bairro   |zona      |
+-----+----------+---------+---------+-------------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+---------------------------+---------+----------+
|3    |[43]      |[43]     |[1]      |[Academia, Churrasqueira, Pla

In [166]:
insight_anuncio = insight_anuncio\
  .withColumn("valor_condominio", col("valores.condominio")
  ).withColumn("valor_iptu", col("valores.iptu")
  ).withColumn("tipo_transacao",concat_ws("," ,col("valores.tipo")) 
  ).withColumn("valor", col("valores.valor")
  ).drop("valores")

insight_anuncio.show(10, False)
insight_anuncio.printSchema()

+-----+----------+---------+---------+-------------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+---------+----------+----------------+----------+--------------+-------+
|andar|area_total|area_util|banheiros|caracteristicas                                                                                                                |id                                  |quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso   |vaga|bairro   |zona      |valor_condominio|valor_iptu|tipo_transacao|valor  |
+-----+----------+---------+---------+-------------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+---------+----------+----------------+----------+--------------+-------

In [168]:
#Alterar o tipo das colunas de string para inteiros
insight_anuncio = insight_anuncio \
  .withColumn("area_total",
              concat_ws("," ,
              col("area_total"))) \
  .withColumn("area_util",
              concat_ws("," ,
              col("area_util"))) \
  .withColumn("banheiros",
              concat_ws("," ,
              col("banheiros"))) \
  .withColumn("quartos",
              concat_ws("," ,
              col("quartos"))) \
  .withColumn("suites",
              concat_ws("," ,
              col("suites"))) \
  .withColumn("vaga",
              concat_ws("," ,
              col("vaga"))) \

insight_anuncio = insight_anuncio \
  .withColumn("area_total" ,
              insight_anuncio["area_total"]
              .cast(IntegerType()))   \
  .withColumn("area_util",
              insight_anuncio["area_util"]
              .cast(IntegerType()))    \
  .withColumn("banheiros"  ,
              insight_anuncio["banheiros"]
              .cast(IntegerType())) \
  .withColumn("quartos"  ,
              insight_anuncio["quartos"]
              .cast(IntegerType())) \
  .withColumn("suites"  ,
              insight_anuncio["suites"]
              .cast(IntegerType())) \
  .withColumn("vaga"  ,
              insight_anuncio["vaga"]
              .cast(IntegerType())) 
insight_anuncio.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: integer (nullable = true)
 |-- area_util: integer (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: integer (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- valor_condominio: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- valor_iptu: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- tipo_transacao: string (nullable = false)
 |-- valor: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [177]:
!git config --global user.email "renatovbastos00@gmail.com"



*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@9a891a4825eb.(none)')
